<a href="https://colab.research.google.com/github/greygrease/tf-stuff/blob/main/09_SkimLit_NLP_Projectipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone Project 2: SkimLit

NLP model to make medical abstracts reading easier

https://arxiv.org/abs/1710.06071 [NN for joint sentence classification]

https://arxiv.org/abs/1612.05251 [PubMed 200k RCT]

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Get data

Download the dataset used in Pubmed https://github.com/Franck-Dernoncourt/pubmed-rct

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 33 (delta 5), reused 5 (delta 5), pack-reused 25
Unpacking objects: 100% (33/33), done.
PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


In [ ]:
# Check what files are in the PubMed_20k dataset
!ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/

dev.txt  test.txt  train.txt


In [ ]:
data_dir = '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/'

import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt']

## Preprocess Data


In [ ]:
# Create func to read the lines of a document

def getlines(filename):
  with open(filename, 'r') as f:
    return f.readlines()

In [ ]:
# Reading Training lines
train_lines = getlines(data_dir + 'train.txt')
train_lines[:5]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n']

In [ ]:
res = [0, 0]
res[0] = res[0] + 1
res

[1, 0]

`1` 2 `3`   

4  `5`  6  

`9` 8 `9`  

In [ ]:
import numpy as np
def sum_diagonals(matrix):
    # your code here
    
    matrix = np.matrix(matrix)
    return sum(np.diag(matrix, 0)), sum(np.diag(np.flip(matrix, axis=1), 0))

In [ ]:
mat = [[1,2,3], 
       [4,5,6], 
       [7,8,9]]

In [ ]:
np.diag(mat, 0)

array([1, 5, 9])

In [ ]:
sum_diagonals(mat)

(8, 15)

In [ ]:
np.flip(mat)

array([[9, 8, 7],
       [6, 5, 4],
       [3, 2, 1]])

In [ ]:
np.flip(mat, axis = 0)

array([[7, 8, 9],
       [4, 5, 6],
       [1, 2, 3]])

In [ ]:
np.flip(mat, axis = 1)

array([[3, 2, 1],
       [6, 5, 4],
       [9, 8, 7]])